In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

vacancy_list = []
ccount = 0
per_page = 50
max_pages = 40

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}
industries = [7, 41, 13, 27, 36, 19, 48, 24, 47, 39, 5, 33, 42, 9, 13, 50, 15, 44, 49]
for industry in industries:
    for page in range(0, max_pages):
        url = f"https://hh.ru/search/vacancy?industry={industry}&page={page}"
        resp = requests.get(url, headers=headers)
        if resp.status_code != 200:
            print(f"Ошибка запроса: {resp.status_code}")
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        vacancies = soup.find_all("div", re.compile(r"^vacancy-info--"))

        if not vacancies:
            break

        for vacancy in vacancies:
            try:

                name_tag = vacancy.find('h2', attrs={'data-qa': 'bloko-header-2'})
                if name_tag:
                    title = name_tag.get_text(strip=True)

                title_tag = vacancy.find('a', attrs={'data-qa': 'serp-item__title'})
                if title_tag:
                    href = title_tag['href']

                addres_tag = vacancy.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-address'})
                if addres_tag:
                    addres = addres_tag.get_text(strip=True)

                price_tag = vacancy.find('div', re.compile(r"^compensation-labels--"))
                if price_tag:
                    price = price_tag.get_text(strip=True)

                vacancy_list.append({
                    'название вакансии': title or '',
                    'зарплата': price or '',
                    'адрес': addres or '',
                    'url': href or ''
                })

            except Exception as e:
              print("Ошибка парсинга вакансии:", e)

        ccount= ccount + len(vacancy_list)
        print(f"Страница {page+1} обработана, всего {ccount} вакансий")

        df = pd.DataFrame(vacancy_list)
        df.to_csv("hh_scraped_vacancies.csv", mode='a', header=not pd.io.common.file_exists("hh_scraped_vacancies.csv"), index=False)
        vacancy_list = []

        time.sleep(1.5)

        if ccount >= 10000:
            break

print("завершено")